# Data Model -  Regression

## Initial Setup for XGBoost

In [1]:
#installation for xgboost
!pip install xgboost 

   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/99.8 MB 1.6 MB/s eta 0:01:02
   ---------------------------------------- 0.2/99.8 MB 2.5 MB/s eta 0:00:40
   ---------------------------------------- 0.5/99.8 MB 2.8 MB/s eta 0:00:36
   ---------------------------------------- 0.6/99.8 MB 3.1 MB/s eta 0:00:33
   ---------------------------------------- 0.7/99.8 MB 2.9 MB/s eta 0:00:35
   ---------------------------------------- 0.9/99.8 MB 3.1 MB/s eta 0:00:32
   ---------------------------------------- 1.1/99.8 MB 3.2 MB/s eta 0:00:31
    --------------------------------------- 1.3/99.8 MB 3.4 MB/s eta 0:00:30
    --------------------------------------- 1.5/99.8 MB 3.4 MB/s eta 0:00:29
    --------------------------------------- 1.7/99.8 MB 3.5 MB/s eta 0:00:29
    ----------------

## Import Libraries

In [4]:
#Importing libraries
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
import sklearn.metrics as metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from sklearn import set_config
set_config(print_changed_only=False)


## Loading dataset 

In [5]:
#Loading Data
train_final = pd.read_csv (r'Datasets\train_final.csv')  

train_final.Date = pd.to_datetime(train_final.Date,format='%Y-%m-%d')
train_final.index = train_final.Date
train_final = train_final.drop('Date', axis=1)
train_final.head()

,Store,Dept,Weekly_Sales,IsHoliday,Type,Size,Year,Week
Date,,,,,,,,
2010-02-05,1,1,24924.50,0,3,151315,2010,5
2010-02-12,1,1,46039.49,1,3,151315,2010,6
2010-02-19,1,1,41595.55,0,3,151315,2010,7
2010-02-26,1,1,19403.54,0,3,151315,2010,8
2010-03-05,1,1,21827.90,0,3,151315,2010,9


## Setting Training Set and Split

In [6]:
train_data, test_data = sklearn.model_selection.train_test_split(train_final, train_size = 0.7,shuffle=False)
X_train, y_train = train_data[['Store', 'Dept','IsHoliday','Type','Size','Year','Week']], train_data['Weekly_Sales']

X_test, y_test = test_data[['Store', 'Dept','IsHoliday','Type','Size','Year','Week']], test_data['Weekly_Sales']

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((295099, 7), (295099,), (126471, 7), (126471,))

## Decision Tree Modeling

In [7]:
#Define Decision Tree
model = DecisionTreeRegressor(random_state=1)

#Setting parameters to test on Decision Tree
param1 = [
    {'min_samples_leaf': range(1,51,2),
     'min_samples_split':range(2,100,2),
     'max_depth':range(5,1000,5)}
]

#Perform Randomized Search CV
grid_search = RandomizedSearchCV(model, param1, cv = 3, verbose = 3, 
                                 n_jobs = -1)
grid_search.fit(X_train, y_train)
results = grid_search.cv_results_
best_param=grid_search.best_params_

#Display the best result
best_param 

Fitting 3 folds for each of 10 candidates, totalling 30 fits


{'min_samples_split': 8, 'min_samples_leaf': 25, 'max_depth': 495}

In [8]:
#Fitting the parameters into Decision Tree Regressor Model
dt = DecisionTreeRegressor(min_samples_split=54,min_samples_leaf=25,max_depth=440,random_state=42)
dt=dt.fit(X_train,y_train)
y_pred_dt = dt.predict(X_test)

#Display the settings
print(dt)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='squared_error', max_depth=440,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_samples_leaf=25,
                      min_samples_split=54, min_weight_fraction_leaf=0.0,
                      monotonic_cst=None, random_state=42, splitter='best')


## Random Forest Regressor Modeling

In [9]:
#Define Random Forest Regressor
model = RandomForestRegressor(random_state = 42)

#Setting parameters to test on Random Forest Regressor Model
params={
 "n_estimators"     : range(100,300,100) ,
 "max_depth"        : [5,25,50,100,200],
 "min_samples_split": [2,5,8,10,15,20],
 "min_samples_leaf" : [1,2,5,8,10]
}

#Perform Randomized Search CV
grid_search = RandomizedSearchCV(model, params, cv = 3, verbose = 3, 
                                 n_jobs = -1)
grid_search.fit(X_train, y_train)
results = grid_search.cv_results_
best_param=grid_search.best_params_

#Display the best result
best_param

Fitting 3 folds for each of 10 candidates, totalling 30 fits


{'n_estimators': 200,
 'min_samples_split': 2,
 'min_samples_leaf': 10,
 'max_depth': 50}

In [10]:
#Random Forest Regressor modeling
rfr = RandomForestRegressor(n_estimators=200,min_samples_split=15, min_samples_leaf=8,max_depth=200)        
rfr.fit(X_train,y_train)
y_pred_rfr=rfr.predict(X_test)

## XGBoost Modeling

In [11]:
#Define XGBoost
model = xgb.XGBRegressor(random_state = 1)

#Setting parameters to test on XGBoost
params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [5,25,50,100,200,500,1000],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]   
}

#Perform Randomized Search CV
grid_search = RandomizedSearchCV(model, params, cv = 3, verbose = 3,n_jobs = -1)
grid_search.fit(X_train, y_train)
results = grid_search.cv_results_
grid_search.best_params_
best_param=grid_search.best_params_
#Display the best result
best_param

Fitting 3 folds for each of 10 candidates, totalling 30 fits


{'min_child_weight': 7,
 'max_depth': 1000,
 'learning_rate': 0.05,
 'gamma': 0.4,
 'colsample_bytree': 0.7}

In [12]:
#XGBoost modeling
xgb=xgb.XGBRegressor(min_child_weight=5,max_depth=25,learning_rate=0.05,gamma=0.1,colsample_bytree=0.7)
xgb.fit(X_train,y_train)
y_pred_xgb=xgb.predict(X_test)